In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [7]:
!pip install langchain langchain-community faiss-cpu sentence-transformers torch transformers datasets pandas tqdm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 73.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

# Load Hugging Face documentation dataset (perfect for AI/ML RAG)
# This dataset contains real HuggingFace API documentation with explanations
dataset = load_dataset("m-ric/huggingface_doc", split="train")

print(f"Dataset size: {len(dataset)} documents")
print(f"Sample document:\n{dataset[0]}")

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

Dataset size: 2647 documents
Sample document:
{'text': ' Create an Endpoint\n\nAfter your first login, you will be directed to the [Endpoint creation page](https://ui.endpoints.huggingface.co/new). As an example, this guide will go through the steps to deploy [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) for text classification. \n\n## 1. Enter the Hugging Face Repository ID and your desired endpoint name:\n\n<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_repository.png" alt="select repository" />\n\n## 2. Select your Cloud Provider and region. Initially, only AWS will be available as a Cloud Provider with the `us-east-1` and `eu-west-1` regions. We will add Azure soon, and if you need to test Endpoints with other Cloud Providers or regions, please let us know.\n\n<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_re

In [3]:
# Convert to DataFrame for easier processing
df = pd.DataFrame({
    'text': [doc['text'] for doc in dataset],
    'source': [doc['source'] for doc in dataset]
})
print(f"\nDataFrame shape: {df.shape}")
print(f"First few sources:\n{df['source'].unique()[:5]}")


DataFrame shape: (2647, 2)
First few sources:
['huggingface/hf-endpoints-documentation/blob/main/docs/source/guides/create_endpoint.mdx'
 'huggingface/evaluate/blob/main/docs/source/choosing_a_metric.mdx'
 'gradio-app/gradio/blob/main/guides/cn/01_getting-started/02_key-features.md'
 'huggingface/transformers/blob/main/docs/source/en/perf_train_tpu_tf.md'
 'gradio-app/gradio/blob/main/demo/blocks_random_slider/run.ipynb']


**Data Preprocessing and Chunking**

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

# Remove documents with missing text
df_clean = df.dropna(subset=['text'])
print(f"Documents after cleaning: {len(df_clean)}")

# Create LangChain Document objects
documents = [
    LangchainDocument(page_content=row['text'], metadata={"source": row['source']})
    for _, row in df_clean.iterrows()
]

# Split documents into chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,           # Optimal size for embeddings
    chunk_overlap=100,         # Overlap for context preservation
    separators=["\n\n", "\n", ".", " "]
)

docs_processed = []
for doc in tqdm(documents[:500], desc="Chunking documents"):  # Use first 500 for demo
    chunks = text_splitter.split_documents([doc])
    docs_processed.extend(chunks)

print(f"Total chunks created: {len(docs_processed)}")
print(f"Sample chunk:\n{docs_processed[0].page_content[:200]}...")

Documents after cleaning: 2647


Chunking documents: 100%|██████████| 500/500 [00:00<00:00, 3426.04it/s]

Total chunks created: 12474
Sample chunk:
Create an Endpoint

After your first login, you will be directed to the [Endpoint creation page](https://ui.endpoints.huggingface.co/new). As an example, this guide will go through the steps to deploy...


**Creating Embeddings using Sentence Transformers**

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Use a lightweight but powerful embedding model
# This model is optimized for semantic similarity
EMBEDDING_MODEL = "all-mpnet-base-v2"

print(f"Loading embedding model: {EMBEDDING_MODEL}")
embedding_model = SentenceTransformer(EMBEDDING_MODEL)

# Embed all chunks
print("Creating embeddings for all chunks...")
chunk_texts = [doc.page_content for doc in docs_processed]
embeddings = embedding_model.encode(
    chunk_texts,
    batch_size=4,
    show_progress_bar=True,
    convert_to_numpy=True
)

print(f"Embeddings shape: {embeddings.shape}")
print(f"Embedding dimension: {embeddings.shape[1]}")

2025-11-18 03:11:11.325052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763435471.785241      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763435471.890508      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating embeddings for all chunks...


Batches:   0%|          | 0/3119 [00:00<?, ?it/s]

Embeddings shape: (12474, 768)
Embedding dimension: 768


**Building FAISS Vector Database**

In [8]:
import faiss
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

# Create HuggingFace embeddings wrapper for LangChain
hf_embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs={"device": "cuda"},  # Use "cuda" if GPU available
    encode_kwargs={"normalize_embeddings": True}  # For cosine similarity
)

# Build FAISS vector database
print("Building FAISS vector database...")
vector_db = FAISS.from_documents(
    docs_processed,
    hf_embeddings,
    distance_strategy=DistanceStrategy.COSINE
)

print("✅ FAISS database created successfully!")

# Save for later use
vector_db.save_local("faiss_index")
print("✅ Database saved locally!")


/tmp/ipykernel_48/3118961586.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(


Building FAISS vector database...
✅ FAISS database created successfully!
✅ Database saved locally!


**Loading LLM for answer generation**

In [9]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
# Use an efficient open-source LLM
LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.1"

print(f"Loading LLM: {LLM_MODEL}")

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Create text generation pipeline
llm_pipeline = pipeline(
    "text-generation",
    model=LLM_MODEL,
    device=1 if device == "cuda" else -1,
    torch_dtype=torch.float16,
    max_new_tokens=128,
    temperature=0.3,# this is for more deterministic answers
    num_beams=1, # Beam search is for better output
    top_p=0.95
)

print("✅ LLM loaded successfully!")


Loading LLM: mistralai/Mistral-7B-Instruct-v0.1
Using device: cuda


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:1


✅ LLM loaded successfully!


In [16]:
# Define RAG prompt template
RAG_PROMPT_TEMPLATE = """
Context:
{context}

Question:
{question}

Answer concisely based only on the above context. If the answer is not found, respond: "I don't know."
"""


**RAG Pipeline**

In [17]:
from typing import Tuple, List

def rag_retrieve_and_generate(
    question: str,
    vector_db,
    llm_pipeline,
    k: int = 5
) -> Tuple[str, List]:
    """
    Complete RAG pipeline: retrieve relevant docs and generate answer
    """
    # Step 1: Retrieve relevant documents
    print(f"🔍 Retrieving documents for: {question}")
    retrieved_docs = vector_db.similarity_search(question, k=k)
    
    # Step 2: Prepare context
    context = "\n".join([
        f"Source {i+1}: {doc.page_content[:300]}...\n(Source: {doc.metadata.get('source', 'Unknown')})\n"
        for i, doc in enumerate(retrieved_docs)
    ])
    
    # Step 3: Build prompt
    prompt = RAG_PROMPT_TEMPLATE.format(
        context=context,
        question=question
    )
    
    # Step 4: Generate answer
    print("💭 Generating answer...")
    response = llm_pipeline(prompt, max_new_tokens=256)
    answer = response[0]['generated_text'].split("Answer:")[-1].strip()
    
    return answer, retrieved_docs

# Test the RAG system
test_question = "What are transformers in machine learning?"
answer, sources = rag_retrieve_and_generate(
    test_question,
    vector_db,
    llm_pipeline,
    k=5
)

print(f"\n{'='*60}")
print(f"Question: {test_question}")
print(f"{'='*60}")
print(f"Answer:\n{answer}")
print(f"{'='*60}")
print(f"Retrieved Sources:")
for i, doc in enumerate(sources):
    print(f"{i+1}. {doc.metadata.get('source', 'Unknown')}")
    print(f"   Content preview: {doc.page_content[:150]}...")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔍 Retrieving documents for: What are transformers in machine learning?
💭 Generating answer...

Question: What are transformers in machine learning?
Answer:
Transformers are a library for machine learning that allows for tasks on several modalities combined, such as table question answering, optical character recognition, information extraction from scanned documents, video classification, and visual question answering. They also include an opinionated design and operations such as tensor contractions.
Retrieved Sources:
1. huggingface/blog/blob/main/lewis-tunstall-interview.md
   Content preview: And if you actually look at what's needed to make this conversion happen in the transformers library, it's fairly gnarly. But we make it so that you o...
2. huggingface/transformers/blob/main/README.md
   Content preview: Transformer models can also perform tasks on **several modalities combined**, such as table question answering, optical character recognition, informa...
3. huggingface/blog/

**Reranking for Quality**

In [18]:
from sentence_transformers import CrossEncoder

# Add reranking to improve retrieval quality
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")

def rag_with_reranking(
    question: str,
    vector_db,
    llm_pipeline,
    reranker,
    k_retrieve: int = 15,
    k_final: int = 5
) -> Tuple[str, List]:
    """
    Advanced RAG with reranking: retrieve more docs, rerank, then generate
    """
    # Retrieve more documents initially
    retrieved_docs = vector_db.similarity_search(question, k=k_retrieve)
    
    # Rerank using cross-encoder
    doc_text_pairs = [(question, doc.page_content) for doc in retrieved_docs]
    scores = reranker.predict(doc_text_pairs)
    
    # Keep top-k after reranking
    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k_final]
    reranked_docs = [retrieved_docs[i] for i in ranked_indices]
    
    # Prepare context and generate answer
    context = "\n".join([
        f"Document {i+1}: {doc.page_content[:400]}...\n"
        for i, doc in enumerate(reranked_docs)
    ])
    
    prompt = RAG_PROMPT_TEMPLATE.format(context=context, question=question)
    response = llm_pipeline(prompt)
    answer = response[0]['generated_text'].split("Answer:")[-1].strip()
    
    return answer, reranked_docs

# Test reranking
answer_reranked, sources_reranked = rag_with_reranking(
    "What is transfer learning in neural networks?",
    vector_db,
    llm_pipeline,
    reranker
)

print(f"Answer (with reranking):\n{answer_reranked}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer (with reranking):
Transfer learning in neural networks is the idea of leveraging the knowledge acquired by a model trained with lots of data on another task. This allows for better results on a new task with less computation, time, and data required.


**Evaluation Metrics**

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_rag_quality(question: str, answer: str, retrieved_docs, embeddings_model):
    """
    Evaluate RAG system quality with metrics
    """
    # Question-Answer relevance
    qa_embedding = embeddings_model.encode(f"{question} {answer}")
    doc_embeddings = embeddings_model.encode([doc.page_content for doc in retrieved_docs])
    
    qa_relevance = cosine_similarity([qa_embedding], doc_embeddings).mean()
    
    # Question-Document relevance
    question_embedding = embeddings_model.encode(question)
    doc_relevance = cosine_similarity([question_embedding], doc_embeddings).mean()
    
    return {
        "qa_relevance": round(qa_relevance, 4),
        "doc_relevance": round(doc_relevance, 4),
        "retrieval_count": len(retrieved_docs)
    }

# Test evaluation
metrics = evaluate_rag_quality(
    "What are transformers?",
    answer,
    sources,
    embedding_model
)

print(f"\n📊 RAG Quality Metrics:")
print(f"  QA Relevance Score: {metrics['qa_relevance']} (higher is better)")
print(f"  Document Relevance: {metrics['doc_relevance']} (higher is better)")
print(f"  Documents Retrieved: {metrics['retrieval_count']}")



📊 RAG Quality Metrics:
  QA Relevance Score: 0.7024999856948853 (higher is better)
  Document Relevance: 0.4977000057697296 (higher is better)
  Documents Retrieved: 5


In [19]:
def interactive_rag_demo():
    """
    Interactive demo for testing RAG system
    """
    print("\n" + "="*60)
    print("🤖 AI/ML Documentation RAG System")
    print("="*60)
    
    test_questions = [
        "How do I use transformers for NLP tasks?",
        "What is transfer learning?",
        "Explain attention mechanisms in detail",
        "How do I fine-tune a pretrained model?",
        "What are embeddings and why are they useful?"
    ]
    
    for question in test_questions:
        print(f"\n📌 Question: {question}")
        print("-" * 60)
        
        answer, retrieved_docs = rag_retrieve_and_generate(
            question,
            vector_db,
            llm_pipeline,
            k=3
        )
        
        metrics = evaluate_rag_quality(question, answer, retrieved_docs, embedding_model)
        
        print(f"✅ Answer: {answer[:300]}...")
        print(f"📊 Relevance Score: {metrics['doc_relevance']}")
        print()

# Run demo
interactive_rag_demo()


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 AI/ML Documentation RAG System

📌 Question: How do I use transformers for NLP tasks?
------------------------------------------------------------
🔍 Retrieving documents for: How do I use transformers for NLP tasks?
💭 Generating answer...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Answer: To use transformers for NLP tasks, you can use the Hugging Face Transformers library. This library provides pre-trained models for a wide range of NLP tasks, as well as tools for training your own models from scratch. You can also use the library to efficiently test large NLP models and integrate Py...
📊 Relevance Score: 0.794700026512146


📌 Question: What is transfer learning?
------------------------------------------------------------
🔍 Retrieving documents for: What is transfer learning?
💭 Generating answer...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Answer: Transfer learning is leveraging the knowledge acquired by a model trained with lots of data on another task....
📊 Relevance Score: 0.6869000196456909


📌 Question: Explain attention mechanisms in detail
------------------------------------------------------------
🔍 Retrieving documents for: Explain attention mechanisms in detail
💭 Generating answer...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Answer: Attention mechanisms in transformer models allow the model to focus on specific parts of the input sequence while processing it. The attention scores are formulated based on the similarity between the query and key vectors, and the resulting attention weights are used to weight the values vector. Th...
📊 Relevance Score: 0.6306999921798706


📌 Question: How do I fine-tune a pretrained model?
------------------------------------------------------------
🔍 Retrieving documents for: How do I fine-tune a pretrained model?
💭 Generating answer...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Answer: To fine-tune a pretrained model, you need to upload "Training Data" in CSV file format. The dataset should be correctly formatted. An example of the required format can be found [here](https://huggingface.co/docs/autotrain/main/en/llm_finetuning). You can also fine-tune a purely pretrained model ins...
📊 Relevance Score: 0.7017999887466431


📌 Question: What are embeddings and why are they useful?
------------------------------------------------------------
🔍 Retrieving documents for: What are embeddings and why are they useful?
💭 Generating answer...
✅ Answer: Embeddings are key for several industrial applications such as Google searches, Snapchat, Facebook, and more. They allow you to build things like chatbots, recommendation systems, zero-shot classifiers, image search, FAQ systems, and more. They are useful because they allow you to match text to text...
📊 Relevance Score: 0.6330000162124634



In [20]:
import json

# Save RAG metadata for reproducibility
rag_metadata = {
    "embedding_model": EMBEDDING_MODEL,
    "llm_model": LLM_MODEL,
    "dataset": "m-ric/huggingface_doc",
    "chunks_created": len(docs_processed),
    "vector_db_type": "FAISS",
    "distance_metric": "cosine"
}

with open("rag_metadata.json", "w") as f:
    json.dump(rag_metadata, f, indent=2)

print("✅ RAG system ready for deployment!")
print(f"📦 Saved files: faiss_index/, rag_metadata.json")

✅ RAG system ready for deployment!
📦 Saved files: faiss_index/, rag_metadata.json
